In [54]:
import pandas as pd
from pypdf import PdfReader
import pdfplumber
import sys
import glob
import numpy as np
import os
import re

### Functions

In [55]:
def extract_text_from_rectangle(pdf_path, page_number, rect):
    with pdfplumber.open(pdf_path) as pdf:
        # Get the specified page
        page = pdf.pages[page_number]
        
        # Define the rectangle (x0, y0, x1, y1)
        x0, y0, x1, y1 = rect
        
        # Extract text from the specified rectangular region
        text = page.within_bbox((x0, y0, x1, y1)).extract_text()
        
        return text

def get_pdf_page_dimensions(pdf_path, page_number=0):
    # Open the PDF file
    reader = PdfReader(pdf_path)

    # Access the specified page
    page = reader.pages[page_number]

    # Get the MediaBox
    media_box = page.get('/MediaBox')
    
    if media_box is not None:
        # Convert the MediaBox values to float
        width = float(media_box[2]) - float(media_box[0])  # x1 - x0
        height = float(media_box[3]) - float(media_box[1])  # y1 - y0

        return width, height
    else:
        return None, None  # Handle case where MediaBox is not found

def is_convertible_to_float(s):
    # Remove commas and whitespace
    s = s.replace(',', '').strip()
    
    # Regular expression pattern for a valid float
    pattern = r'^[-+]?(\d+\.?\d*|\.\d+)([eE][-+]?\d+)?$'
    
    # Check if the string matches the pattern
    if re.match(pattern, s):
        return True
    else:
        return False

def extract_dairy_info(pdf_path, page_number, rectangle, name, dictionary):
    # Extract text from rectangle
    text = extract_text_from_rectangle(pdf_path, page_number, rectangle)

    # If looking at cow types, different method
    if name == "Herd Info":
        # Split the string by spaces to get a list of string numbers
        text = text.replace(",", "")
        number_list = text.split()
        # Convert the list of string numbers to a list of integers (or floats if needed)
        number_list = [int(num) for num in number_list]
        # Assign the values to the respective variables
        cow_types = ["Milk Cows", "Dry Cows", "Bred Heifers", "Heifers", "Calves (4-6 mo.)", "Calves (0-3 mo.)"]
        if len(number_list) != len(cow_types):
            print("Error: Number of cow counts does not match number of cow types.")
            raise ValueError()
            return
        else:
            for i in range(len(cow_types)):
                dictionary[cow_types[i]] = number_list[i]
            return
        return
    
    # If getting float value, different method
    if name in ["Total Manure Excreted (tons)","Total Process Wastewater Generated (gals)","Total Dry Manure Generated N (lbs)","Total Dry Manure Generated N After Ammonia Losses (lbs)","Average Milk Production (lb per cow per day)",
                "Total Dry Manure Generated P (lbs)", "Total Dry Manure Generated K (lbs)", "Total Dry Manure Generated Salt (lbs)",
                "Total Process Wastewater Generated (gals)", "Total Process Wastewater Generated N (lbs)", "Total Process Wastewater Generated P (lbs)", "Total Process Wastewater Generated K (lbs)", "Total Process Wastewater Generated Salt (lbs)"]:
        # If text is empty, assign 0
        if is_convertible_to_float(text) == False:
            dictionary[name] = 0
            return
        # remove any commas and convert to float
        else:
            text = text.replace(",", "")
            text = float(text)
            dictionary[name] = text
            return

    # For text info just add the text to the dictionary      
    else:
        # Assign the value to the dictionary
        dictionary[name] = text
        return

def extract_nutrient_application_info(pdf_path, dictionary):
    data_order =  ["Applied N Dry Manure (lbs)", "Applied P Dry Manure (lbs)", "Applied K Dry Manure (lbs)", "Applied Salt Dry Manure (lbs)",
                   "Applied Process Wastewater N (lbs)", "Applied Process Wastewater P (lbs)", "Applied Process Wastewater K (lbs)", "Applied Process Wastewater Salt (lbs)",
                   "Applied to Remove Ratio N", "Applied to Remove Ratio P", "Applied to Remove Ratio K", "Applied to Remove Ratio Salt"]
                    #"Total Dry Manure Generated N (lbs)"]
                    
    with pdfplumber.open(pdf_path) as pdf:
        page_number = -1
        # Find relevant page
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if "SUMMARY OF NUTRIENT APPLICATIONS" in text:
                page_number = i
                break
        if page_number != -1:
            #print(f'"SUMMARY OF NUTRIENT APPLICATIONS" found on page {page_number + 1}')
            # Define the coordinates of the nutrient balance and applied to removed ratio data
            # Coordinates are in the format (x0, top, x1, bottom)
            x0, top, x1, bottom = 150, 175, 600, 210
            
            # Extract the page where the string was found
            page = pdf.pages[page_number]
            
            # Extract text within the defined rectangle
            cropped_text = page.within_bbox((x0, top, x1, bottom)).extract_text()

            # Convert the extracted text to a list of floats
            l1 = convert_to_float_list(cropped_text)

            # Next, extract the applied to removed ratio data
            x0, top, x1, bottom = 150, 290, 600, 320
            cropped_text = page.within_bbox((x0, top, x1, bottom)).extract_text()
            l2 = convert_to_float_list(cropped_text)

            # Merge lists
            l = l1 + l2

            # Next, extract the Dry Manure Total N (lbs)
            #x0, top, x1, bottom = 180, 170, 300, 190
            #cropped_text = page.within_bbox((x0, top, x1, bottom)).extract_text()
            #l2 = convert_to_float_list(cropped_text)
            #l = l + l2

            # Next assign the values to the dictionary
            if len(l) != len(data_order):
                print("Error: Number of nutrient values does not match number of nutrient types.")
                raise ValueError()
                return
            for i in range(len(data_order)):
                dictionary[data_order[i]] = l[i]
        else:
            print('"SUMMARY OF NUTRIENT APPLICATIONS" not found in the entire PDF.')

def extract_text_to_the_right_of_phrase(page, phrase):
    text = page.extract_text()
    if text:
        lines = text.split('\n')
        for line in lines:
            if phrase in line:
                # This assumes that text to the right is separated by whitespace
                # Adjust the splitting logic if needed based on your document's structure
                parts = line.split(phrase)
                if len(parts) > 1:
                    right_text = parts[1].strip()
                    return right_text
    return None

def extract_nutrient_total_exports(pdf_path, dictionary):                    
    with pdfplumber.open(pdf_path) as pdf:
        page_number = -1
        # Find relevant page
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if "Total exports for all materials" in text:
                page_number = i
                break
        if page_number != -1:
            # Extract the text to the right of where the string was found
            #print(f'"Total exports for all materials" found on page {page_number + 1}')
            text = extract_text_to_the_right_of_phrase(pdf.pages[page_number], "Total exports for all materials")
            # Convert to list of floats
            l = convert_to_float_list(text)
            # Take first element
            N_exports = l[0]
            P_exports = l[1]
            K_exports = l[2]
            Salt_exports = l[3]
            # Assign the values to the dictionary
            dictionary["Total Exports N (lbs)"] = N_exports
            dictionary["Total Exports P (lbs)"] = P_exports
            dictionary["Total Exports K (lbs)"] = K_exports
            dictionary["Total Exports Salt (lbs)"] = Salt_exports

        else:
            dictionary["Total Exports N (lbs)"] = 0
            dictionary["Total Exports P (lbs)"] = 0
            dictionary["Total Exports K (lbs)"] = 0
            dictionary["Total Exports Salt (lbs)"] = 0

            print('"Total exports for all materials" not found in the entire PDF.')

def convert_to_float_list(text):
    # Remove any unwanted characters and split the text by whitespace
    components = re.split(r'\s+', text.strip())
    
    float_numbers = []
    for component in components:
        # Remove commas used as thousands separators
        cleaned_component = component.replace(',', '')

        try:
            # Convert the cleaned component to a float and append to the list
            float_numbers.append(float(cleaned_component))
        except ValueError:
            # Handle the case where the conversion fails (if any)
            print(f"Could not convert '{component}' to float")
    
    return float_numbers

### Extract Herd, Manure, Wastewater, Nitrogen Info

In [56]:
# Set folder and output name
folder = "/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms"
output_folder = "/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Results"
name = "TulareWest_2023_R5-2007-0035"

# Use glob to find all PDF files in the directory
pdf_files = glob.glob(os.path.join(folder, '*.pdf'))
# Sort the list of files alphabetically
pdf_files.sort()

# Create a pandas dataFrame to store the results
df = pd.DataFrame(columns=["Dairy Name", "Dairy Address", "Milk Cows", "Dry Cows", "Bred Heifers", "Heifers", "Calves (4-6 mo.)", "Calves (0-3 mo.)", "Total Herd Size",
                            "Total Manure Excreted (tons)", "Total Dry Manure Generated N (lbs)", "Total Dry Manure Generated N After Ammonia Losses (lbs)",
                            "Total Dry Manure Generated P (lbs)", "Total Dry Manure Generated K (lbs)", "Total Dry Manure Generated Salt (lbs)",
                            "Total Process Wastewater Generated (gals)", "Total Process Wastewater Generated N (lbs)", "Total Process Wastewater Generated P (lbs)", "Total Process Wastewater Generated K (lbs)", "Total Process Wastewater Generated Salt (lbs)",
                            "Average Milk Production (lb per cow per day)",
                            #"Total Dry Manure Generated N (lbs)", 
                            "Applied N Dry Manure (lbs)", "Applied P Dry Manure (lbs)", "Applied K Dry Manure (lbs)", "Applied Salt Dry Manure (lbs)",
                            "Applied Process Wastewater N (lbs)", "Applied Process Wastewater P (lbs)", "Applied Process Wastewater K (lbs)", "Applied Process Wastewater Salt (lbs)",
                            "Applied to Remove Ratio N", "Applied to Remove Ratio P", "Applied to Remove Ratio K", "Applied to Remove Ratio Salt",
                            "Total Exports N (lbs)", "Total Exports P (lbs)", "Total Exports K (lbs)", "Total Exports Salt (lbs)"])



# Print out the list of PDF files
for pdf_path in pdf_files:
    print(pdf_path)
    # Create dictionary to store all results for each dairy
    dairy_dict = dict()
    # Get Dairy Name
    extract_dairy_info(pdf_path, 0, (287, 85, 500, 130), "Dairy Name", dairy_dict)
    # Get Dairy Address
    extract_dairy_info(pdf_path, 0, (50, 120, 775, 140), "Dairy Address", dairy_dict)

    # Get Dairy Info (Sometimes on 2nd, sometimes 3rd page)
    try:
        # Get Herd Type
        extract_dairy_info(pdf_path, 1, (125, 170, 775, 190), "Herd Info", dairy_dict)
        # Get Total Manure Excreted
        extract_dairy_info(pdf_path, 1, (188, 250, 262, 269), "Total Manure Excreted (tons)", dairy_dict)
        # Get Average Milk Production (lb per cow per day)
        extract_dairy_info(pdf_path, 1, (147, 215, 238, 230), "Average Milk Production (lb per cow per day)", dairy_dict)
        # Get N from manure after ammonia losses
        extract_dairy_info(pdf_path, 1, (539, 260, 608, 285), "Total Dry Manure Generated N After Ammonia Losses (lbs)", dairy_dict)
        # Get Total N Dry Manure
        extract_dairy_info(pdf_path, 1, (188, 260, 262, 285), "Total Dry Manure Generated N (lbs)", dairy_dict)
        # Get P from manure
        extract_dairy_info(pdf_path, 1, (188, 280, 262, 300), "Total Dry Manure Generated P (lbs)", dairy_dict)
        # Get K from manure
        extract_dairy_info(pdf_path, 1, (188, 300, 262, 315), "Total Dry Manure Generated K (lbs)", dairy_dict)
        # Get Salt from manure
        extract_dairy_info(pdf_path, 1, (188, 315, 262, 345), "Total Dry Manure Generated Salt (lbs)", dairy_dict)
        # Get Process Wastewater Generated
        extract_dairy_info(pdf_path, 1, (182, 350, 247, 370), "Total Process Wastewater Generated (gals)", dairy_dict)
        # Get Process Wastewater N
        extract_dairy_info(pdf_path, 1, (182, 370, 247, 388), "Total Process Wastewater Generated N (lbs)", dairy_dict)
        # Get Process Wastewater P
        extract_dairy_info(pdf_path, 1, (182, 385, 247, 402), "Total Process Wastewater Generated P (lbs)", dairy_dict)
        # Get Process Wastewater K
        extract_dairy_info(pdf_path, 1, (182, 395, 247, 417), "Total Process Wastewater Generated K (lbs)", dairy_dict)
        # Get Process Wastewater Salt
        extract_dairy_info(pdf_path, 1, (182, 412, 247, 435), "Total Process Wastewater Generated Salt (lbs)", dairy_dict)


    except ValueError:
        # Get Herd Type
        extract_dairy_info(pdf_path, 2, (125, 170, 775, 190), "Herd Info", dairy_dict)
        # Get Total Manure Excreted
        extract_dairy_info(pdf_path, 2, (188, 250, 262, 269), "Total Manure Excreted (tons)", dairy_dict)
        # Get Average Milk Production (lb per cow per day)
        extract_dairy_info(pdf_path, 2, (147, 215, 238, 230), "Average Milk Production (lb per cow per day)", dairy_dict)
        # Get N from manure after ammonia losses
        extract_dairy_info(pdf_path, 2, (539, 260, 608, 285), "Total Dry Manure Generated N After Ammonia Losses (lbs)", dairy_dict)
        # Get Total N Dry Manure
        extract_dairy_info(pdf_path, 2, (188, 260, 262, 285), "Total Dry Manure Generated N (lbs)", dairy_dict)
        # Get P from manure
        extract_dairy_info(pdf_path, 2, (188, 280, 262, 300), "Total Dry Manure Generated P (lbs)", dairy_dict)
        # Get K from manure
        extract_dairy_info(pdf_path, 2, (188, 300, 262, 315), "Total Dry Manure Generated K (lbs)", dairy_dict)
        # Get Salt from manure
        extract_dairy_info(pdf_path, 2, (188, 315, 262, 345), "Total Dry Manure Generated Salt (lbs)", dairy_dict)
        # Get Process Wastewater Generated
        extract_dairy_info(pdf_path, 2, (182, 350, 247, 370), "Total Process Wastewater Generated (gals)", dairy_dict)
        # Get Process Wastewater N
        extract_dairy_info(pdf_path, 2, (182, 370, 247, 388), "Total Process Wastewater Generated N (lbs)", dairy_dict)
        # Get Process Wastewater P
        extract_dairy_info(pdf_path, 2, (182, 385, 247, 402), "Total Process Wastewater Generated P (lbs)", dairy_dict)
        # Get Process Wastewater K
        extract_dairy_info(pdf_path, 2, (182, 395, 247, 417), "Total Process Wastewater Generated K (lbs)", dairy_dict)
        # Get Process Wastewater Salt
        extract_dairy_info(pdf_path, 2, (182, 412, 247, 435), "Total Process Wastewater Generated Salt (lbs)", dairy_dict)


    # Get Nutrient Application Info
    try:
        extract_nutrient_application_info(pdf_path, dairy_dict)

    except ValueError:
        print("Error: Could not find Nutrient Application Info")
        sys.exit()
    
    # Get Total Nutrient Exports Info
    try:
        extract_nutrient_total_exports(pdf_path, dairy_dict)
    except ValueError:
        print("Error: Could not find Total Exports of N Info")
        sys.exit()

    #print(dairy_dict)
    print('****************')

    # Convert the dictionary to a DataFrame
    new_row_df = pd.DataFrame([dairy_dict])

    # Concatenate the new DataFrame with the original DataFrame
    df = pd.concat([df, new_row_df], ignore_index=True)
display(df)


/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms/2023AR_4K Dairy Farm Partnership_7976 Avenue 84_W Tulare.pdf
****************
/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms/2023AR_AC Enterprises Dairy_15499 Road 72_W Tulare.pdf


/var/folders/27/x2_s1xjn34l8k5_bc5sj73jr0000gn/T/ipykernel_89998/1205658095.py:116: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row_df], ignore_index=True)


****************
/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms/2023AR_Adriano Nunes & Son Dairy Farming_18675 Road 32_W Tulare.pdf
"Total exports for all materials" not found in the entire PDF.
****************
/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms/2023AR_Airoso Dairy_18809 Road 64_W Tulare.pdf
"Total exports for all materials" not found in the entire PDF.
****************
/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms/2023AR_Aukeman Farms #2_18193 South I Drive_W Tulare.pdf
"Total exports for all materials" not found in the entire PDF.
****************
/Users/ianbick/Library/CloudStorage/OneDrive-Stanford/CAFO/CAFO_Water_Reports/Region 5/Tulare West Dairy/Correct_Forms/2023AR_Aukeman Farms Dairy_17297 Road 96_W Tulare.pdf
****************
/Users/ianbick/Library/C

,Dairy Name,Dairy Address,Milk Cows,Dry Cows,Bred Heifers,Heifers,Calves (4-6 mo.),Calves (0-3 mo.),Total Herd Size,Total Manure Excreted (tons),...,Applied Process Wastewater K (lbs),Applied Process Wastewater Salt (lbs),Applied to Remove Ratio N,Applied to Remove Ratio P,Applied to Remove Ratio K,Applied to Remove Ratio Salt,Total Exports N (lbs),Total Exports P (lbs),Total Exports K (lbs),Total Exports Salt (lbs)
0,4K Dairy Family Partnership,7976 84 AVE Pixley Tulare 93256,2598,271,925,943,245,955,NaN,94129.38,...,294800.98,2133457.77,1.21,1.11,1.23,3.36,200083.05,80033.22,207653.76,0.00
1,AC Enterprises,15499 Road 72 Tipton Tulare 93272,488,110,321,148,69,89,NaN,20409.01,...,124874.58,915363.05,1.10,0.32,1.08,5.96,76965.36,25830.84,66158.58,0.00
2,Adriano Nunes & Son Dairy-Farming,18675 32 RD Tulare Tulare 93274,790,144,295,300,140,109,NaN,28885.28,...,7616.32,89601.02,0.80,1.09,0.68,0.28,0.00,0.00,0.00,0.00
3,Airoso Dairy,18809 Road 64 Tulare Tulare 93274,0,115,150,135,70,0,NaN,4293.01,...,7655.94,49937.82,0.74,1.00,0.61,1.59,0.00,0.00,0.00,0.00
4,Aukeman Farms #2 (Formerly Golden Valley Dairy,18183 S I DR Tulare Tulare 93274,0,0,0,65,125,0,NaN,1038.18,...,0.00,0.00,1.28,1.87,1.28,0.95,0.00,0.00,0.00,0.00
5,Aukeman Farms,17297 Road 96 Tulare Tulare 93274,3140,415,1935,840,640,535,NaN,119861.08,...,278308.56,1465621.75,1.23,1.08,0.76,2.05,323759.19,85994.79,392281.99,0.00
6,Channel Islands Dairy Farms,13406 Road 24 Corcoran Tulare 93212,8754,1062,2725,1875,875,0,NaN,276279.22,...,235316.16,1646231.25,1.24,0.89,1.83,1.95,1092397.70,342635.57,1298130.50,5010424.35
7,"Coronado Dairy Farms, LLC",5850 160 AVE Tipton Tulare 93272,4224,707,518,0,0,0,NaN,125008.96,...,427704.86,3405153.86,1.26,1.23,1.31,2.36,0.00,0.00,0.00,0.00
8,Cow Palace Dairy,5595 96 AVE Pixley Tulare 93256,0,0,0,0,0,0,NaN,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Curti Family Farms,19493 Road 30 Tulare Tulare 93274,600,0,0,0,0,0,NaN,14682.43,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Calculations

In [57]:
# Convert all numeric fields to numeric
columns_to_convert = ['Total Manure Excreted (tons)', 'Total Process Wastewater Generated (gals)', 'Total Dry Manure Generated N (lbs)','Total Dry Manure Generated N After Ammonia Losses (lbs)','Average Milk Production (lb per cow per day)',
                    "Total Dry Manure Generated P (lbs)", "Total Dry Manure Generated K (lbs)", "Total Dry Manure Generated Salt (lbs)",
                    "Total Exports N (lbs)", "Total Exports P (lbs)", "Total Exports K (lbs)", "Total Exports Salt (lbs)",
                    "Total Process Wastewater Generated (gals)", "Total Process Wastewater Generated N (lbs)", "Total Process Wastewater Generated P (lbs)", "Total Process Wastewater Generated K (lbs)", "Total Process Wastewater Generated Salt (lbs)"]

# Convert specified columns to numeric
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

##########################################################
# Calculate Total Size of Herd
##########################################################
df["Total Herd Size"] = df["Milk Cows"] + df["Dry Cows"] + df["Bred Heifers"] + df["Heifers"] + df["Calves (4-6 mo.)"] + df["Calves (0-3 mo.)"]

##########################################################
# Sum Nutrients from Dry Manure and Process Wastewater
##########################################################
df["Total Applied N (lbs)"] = df["Applied N Dry Manure (lbs)"] + df["Applied Process Wastewater N (lbs)"]
df["Total Applied P (lbs)"] = df["Applied P Dry Manure (lbs)"] + df["Applied Process Wastewater P (lbs)"]
df["Total Applied K (lbs)"] = df["Applied K Dry Manure (lbs)"] + df["Applied Process Wastewater K (lbs)"]
df["Total Applied Salt (lbs)"] = df["Applied Salt Dry Manure (lbs)"] + df["Applied Process Wastewater Salt (lbs)"]

##########################################################
# Sum Nutrients from Dry Manure and Process Wastewater
##########################################################
df["Unaccounted-for N (lbs)"] = df["Total Dry Manure Generated N After Ammonia Losses (lbs)"] + df["Total Process Wastewater Generated N (lbs)"]  - df["Total Applied N (lbs)"] - df["Total Exports N (lbs)"]
df["Unaccounted-for P (lbs)"] = df["Total Dry Manure Generated P (lbs)"] + df["Total Process Wastewater Generated P (lbs)"] - df["Total Applied P (lbs)"] - df["Total Exports P (lbs)"]
df["Unaccounted-for K (lbs)"] = df["Total Dry Manure Generated K (lbs)"] + df["Total Process Wastewater Generated K (lbs)"] - df["Total Applied K (lbs)"] - df["Total Exports K (lbs)"]
df["Unaccounted-for Salt (lbs)"] = df["Total Dry Manure Generated Salt (lbs)"] + df["Total Process Wastewater Generated Salt (lbs)"] - df["Total Applied Salt (lbs)"] - df["Total Exports Salt (lbs)"]



##########################################################
## Calculate total Reported Nutrients
##########################################################
#df["Total Reported N (lbs)"] = df["Total Dry Manure Generated N After Ammonia Losses (lbs)"] + df["Total Process Wastewater Generated N (lbs)"]
#df["Total Reported P (lbs)"] = df["Total Dry Manure Generated P (lbs)"] + df["Total Process Wastewater Generated P (lbs)"]
#df["Total Reported K (lbs)"] = df["Total Dry Manure Generated K (lbs)"] + df["Total Process Wastewater Generated K (lbs)"]
#df["Total Reported Salt (lbs)"] = df["Total Dry Manure Generated Salt (lbs)"] + df["Total Process Wastewater Generated Salt (lbs)"]

##########################################################
### Calculate Milk Production
##########################################################
# Milk pounds to liters 
# (https://books.lib.uoguelph.ca/dairyscienceandtechnologyebook/chapter/physical-properties-of-milk/#:~:text=With%20all%20of%20this%20in,m3%20at%2020°%20C.)

lb_to_kg = 0.453592 # kg/lb
kg_to_L = 0.971 # L/kg at 20 degrees C

def calculateAnnualMilkProduction(x,lb_to_kg,kg_to_L):
    try:
        # Calculate Average Milk Production (kg per cow)
        x_kg = x['Average Milk Production (lb per cow per day)'] * lb_to_kg
        # Convert to liters per cow
        x_l = x_kg * kg_to_L
        # Convert to total annual milk production in L
        x_l_annual = x_l * (x['Milk Cows']+x['Dry Cows']) * 365
        return x_kg, x_l, x_l_annual
    except Exception as e:
        print(e)
        return np.nan, np.nan, np.nan

df[['Average Milk Production (kg per cow)', 'Average Milk Production (L per cow)', 'Total Annual Milk Production (L)']] = df.apply(lambda x: calculateAnnualMilkProduction(x,lb_to_kg, kg_to_L),axis=1, result_type='expand')




### Calculate liters of wastewater and ratio to milk production
def calculateWastewater(x):
    # Check if wastewater is generated
    if x['Total Process Wastewater Generated (gals)'] == 0 or pd.isna(x['Total Process Wastewater Generated (gals)']):
        return 0, np.nan
    else:
        # Convert to liters
        x_l = x['Total Process Wastewater Generated (gals)'] * 3.78541
        try:
            # Calculate ratio of wastewater to milk production
            ratio = x_l / x['Total Annual Milk Production (L)']
            return x_l, ratio
        except Exception as e:
            print(e)
            return x_l, np.nan

df[["Total Process Wastewater Generated (L)", "Ratio of Wastewater to Milk (L/L)"]] = df.apply(lambda x: calculateWastewater(x),axis=1, result_type='expand')



### Calculate Nitrogen from Manure (USDA)
# Convert tons of manure to N
# 12.92 Pounds of nitrogen/ton wet weight manure 
# https://www.nrcs.usda.gov/sites/default/files/2022-10/ManRpt_KelMofGol_2007_final.pdf
def calculateUsdaNitrogenFromManure(x):
    if x["Total Herd Size"] == 0:
        return np.nan, np.nan
    else:
        try:
            x_n_usda = x['Total Manure Excreted (tons)'] * 12.92
            x_ratio = x_n_usda / x["Total Dry Manure Generated N (lbs)"]
            return x_n_usda, x_ratio
        except Exception as e:
            print(e)
            return np.nan, np.nan

df[['USDA Nitrogen from Manure (lbs)',"Ratio of USDA N to Reported N"]] = df.apply(lambda x: calculateUsdaNitrogenFromManure(x),axis=1, result_type='expand')




### Calculate Nitrogen from Manure (UCCE)
def calculateUcceNitrogenFromManure(x):
    if x["Total Herd Size"] == 0:
        return np.nan, np.nan
    else:
        try:
            x_n_ucce = ((x['Milk Cows'] + x['Dry Cows']) + ((x['Bred Heifers'] + x['Heifers'])*(1.5/4.1)) + ((x['Calves (4-6 mo.)'] + x['Calves (0-3 mo.)'])*(0.5/4.1))) * 365
            x_ratio = x_n_ucce / x["Total Dry Manure Generated N (lbs)"]
            return x_n_ucce, x_ratio
        except Exception as e:
            print(e)
            return np.nan, np.nan

df[['UCCE Nitrogen from Manure (lbs)',"Ratio of UCCE N to Reported N"]] = df.apply(lambda x: calculateUcceNitrogenFromManure(x),axis=1, result_type='expand')





### Calculate Manure Conversion Factor
def calculateManureConversionFactor(x):
    try:
        x_mcf = x["Total Manure Excreted (tons)"] / ( (x['Milk Cows'] + x['Dry Cows']) + ((x['Bred Heifers'] + x['Heifers'])*(1.5/4.1)) + ((x['Calves (4-6 mo.)'] + x['Calves (0-3 mo.)'])*(1.5/4.1)))
        return x_mcf
    except Exception as e:
        print(e)
        return np.nan
        

df['Manure Conversion Factor (tons per cow per year)'] = df.apply(lambda x: calculateManureConversionFactor(x),axis=1)




### Calculate Unaccounted-for Nitrogen
#def calculateUnaccountedForNitrogen(x):
#    try:
#        x_un = x["Total Dry Manure Generated N After Ammonia Losses (lbs)"] -  x["Total Exports N (lbs)"] - x["Total Dry Manure Generated N (lbs)"]
#        return x_un
#    except Exception as e:
#        print(e)
#        return np.nan
#
#df['Unaccounted for Nitrogen (lbs)'] = df.apply(lambda x: calculateUnaccountedForNitrogen(x),axis=1)


display(df)
#df.to_csv(output_folder + "/" + name + ".csv", index=False)


float division by zero
float division by zero
float division by zero
float division by zero


,Dairy Name,Dairy Address,Milk Cows,Dry Cows,Bred Heifers,Heifers,Calves (4-6 mo.),Calves (0-3 mo.),Total Herd Size,Total Manure Excreted (tons),...,Average Milk Production (kg per cow),Average Milk Production (L per cow),Total Annual Milk Production (L),Total Process Wastewater Generated (L),Ratio of Wastewater to Milk (L/L),USDA Nitrogen from Manure (lbs),Ratio of USDA N to Reported N,UCCE Nitrogen from Manure (lbs),Ratio of UCCE N to Reported N,Manure Conversion Factor (tons per cow per year)
0,4K Dairy Family Partnership,7976 84 AVE Pixley Tulare 93256,2598,271,925,943,245,955,5937,94129.38,...,32.658624,31.711524,3.320783e+07,3.601992e+08,10.846816,1.216152e+06,1.041813,1.350046e+06,1.156514,23.582818
1,AC Enterprises,15499 Road 72 Tipton Tulare 93272,488,110,321,148,69,89,1225,20409.01,...,38.555320,37.437216,8.171421e+06,2.195538e+08,26.868494,2.636844e+05,1.065156,2.879316e+05,1.163103,24.666728
2,Adriano Nunes & Son Dairy-Farming,18675 32 RD Tulare Tulare 93274,790,144,295,300,140,109,1778,28885.28,...,29.483480,28.628459,9.759728e+06,1.466657e+07,1.502764,3.731978e+05,1.053164,4.314478e+05,1.217546,23.242463
3,Airoso Dairy,18809 Road 64 Tulare Tulare 93274,0,115,150,135,70,0,470,4293.01,...,0.453592,0.440438,1.848738e+04,2.852685e+07,1543.044653,5.546569e+04,1.074687,8.314878e+04,1.611067,17.531216
4,Aukeman Farms #2 (Formerly Golden Valley Dairy,18183 S I DR Tulare Tulare 93274,0,0,0,65,125,0,190,1038.18,...,0.453592,0.440438,0.000000e+00,0.000000e+00,NaN,1.341329e+04,1.068277,1.424390e+04,1.134430,14.935221
5,Aukeman Farms,17297 Road 96 Tulare Tulare 93274,3140,415,1935,840,640,535,7505,119861.08,...,34.019400,33.032837,4.286258e+07,2.543398e+08,5.933842,1.548605e+06,1.048969,1.720441e+06,1.165364,23.971631
6,Channel Islands Dairy Farms,13406 Road 24 Corcoran Tulare 93212,8754,1062,2725,1875,875,0,15291,276279.22,...,28.122704,27.307146,9.783713e+07,6.749014e+08,6.898213,3.569528e+06,1.018839,4.236056e+06,1.209085,23.375758
7,"Coronado Dairy Farms, LLC",5850 160 AVE Tipton Tulare 93272,4224,707,518,0,0,0,5449,125008.96,...,32.658624,31.711524,5.707488e+07,3.373375e+08,5.910437,1.615116e+06,1.004554,1.868987e+06,1.162454,24.413370
8,Cow Palace Dairy,5595 96 AVE Pixley Tulare 93256,0,0,0,0,0,0,0,1.00,...,31.751440,30.830648,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
9,Curti Family Farms,19493 Road 30 Tulare Tulare 93274,600,0,0,0,0,0,600,14682.43,...,27.215520,26.426270,5.787353e+06,0.000000e+00,NaN,1.896970e+05,0.987896,2.190000e+05,1.140498,24.470717


In [58]:
print(df[["Total Process Wastewater Generated (gals)","Total Process Wastewater Generated (L)"]])

    Total Process Wastewater Generated (gals)  \
0                                  95154618.0   
1                                  58000000.0   
2                                   3874500.0   
3                                   7536000.0   
4                                         0.0   
5                                  67189500.0   
6                                 178290180.0   
7                                  89115173.0   
8                                         0.0   
9                                         0.0   
10                                 49793291.0   
11                                 80683966.0   
12                                 54021953.0   
13                                        0.0   
14                                 17670900.0   
15                                 26116304.0   
16                                 76558021.0   
17                                 99216000.0   
18                                  1788000.0   
19                  

In [59]:
# Export csv
df.to_csv(output_folder + "/" + name + "_N_P_K_Salt_Balance_Ratios" + ".csv", index=False)